In [1]:
import pyptvgtfs
import pandas as pd
import os

In [2]:
VERSIONS = ['20220403_025040',
'20220715_191941',
'20230805_030129',
'20231202_053358',
'20240229_224711',]

In [3]:
DFS = [pyptvgtfs.process_gtfs_zip(f'../downloads/{f}/gtfs.zip', f) for f in VERSIONS]
# Per file: 40s - 1m - 3m. 5 files: 2m - 5m

In [4]:
DFK = pd.concat(DFS, axis=0).set_index(['version_id', 'branch_id', 'table_name'])['df'].to_dict()

In [13]:
VERSIONS_2023 = ['20231021_105623',
'20231021_105623',
'20231030_224547',
'20231107_012112',
'20231113_155410',
'20231122_000348',
'20231130_234004',]
DFS_2023 = [pyptvgtfs.process_gtfs_zip(f'../downloads/{f}/gtfs.zip', f) for f in VERSIONS_2023]
# Per file: 40s - 1m - 3m. 5 files: 2m - 5m. 7 files: 3m - 5sm

In [22]:
VERSIONS_2023_24 = ['20231225_101846', '20240130_030040']
DFS_2023_24 = [pyptvgtfs.process_gtfs_zip(f'../downloads/{f}/gtfs.zip', f) for f in VERSIONS_2023_24]
# Per file: 40s - 1m - 3m. 5 files: 2m - 5m. 7 files: 3m - 5sm. 2 files: 1m 30s - 2m

In [58]:
DFK : dict[tuple, pd.DataFrame] = pd.concat(DFS_2023 + DFS + DFS_2023_24, axis=0).set_index(['version_id', 'branch_id', 'table_name'])['df'].to_dict()
VERSIONS = sorted(list(set(VERSIONS_2023 + VERSIONS + VERSIONS_2023_24)))

In [61]:
DFK['20230805_030129', '4', 'trips'].head()

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
0,12-831-aus-2,T2_1,12-831--2-T2-1,12-831-aus-2.1.H,Kingsmere Estate,0
1,12-831-aus-2,T2_1,12-831--2-T2-10,12-831-aus-2.1.H,Kingsmere Estate,0
2,12-831-aus-2,T2_1,12-831--2-T2-11,12-831-aus-2.1.H,Kingsmere Estate,0
3,12-831-aus-2,T2_1,12-831--2-T2-12,12-831-aus-2.1.H,Kingsmere Estate,0
4,12-831-aus-2,T2_1,12-831--2-T2-13,12-831-aus-2.1.H,Kingsmere Estate,0


In [62]:
DFK['20231021_105623', '4', 'trips'].head()

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
0,12-831-aus-1,MF1-12-831-aus,12-831--1-MF1-102,12-831-aus-1.3.H,Kingsmere Estate,0
1,12-831-aus-1,MF1-12-831-aus,12-831--1-MF1-104,12-831-aus-1.3.H,Kingsmere Estate,0
2,12-831-aus-1,MF1-12-831-aus,12-831--1-MF1-106,12-831-aus-1.3.H,Kingsmere Estate,0
3,12-831-aus-1,MF1-12-831-aus,12-831--1-MF1-108,12-831-aus-1.3.H,Kingsmere Estate,0
4,12-831-aus-1,MF1-12-831-aus,12-831--1-MF1-110,12-831-aus-1.3.H,Kingsmere Estate,0


In [ ]:
# It seems that after at least August 2023 and since at least October 2023, the service_id for buses ('4') has changed from normal service names like all others (T2_1) to specific service names to each route (MF1-12-831-aus). For trip id, before, it's 2-831--2-T2-1, now, it's 12-831--1-MF1-102. Seems like the format of trip_ids in 4 (use '-' instead of '.') being different from other operation branches (1,2...) is still consistent among versions.
# On one hand this change makes 'calendar' and 'calendar_dates' tables become longer, have more redundant data points, and the service_id becomes longer. On the other hand, more specific service patterns to each route can be more informative and useful for table joining or querying about the service time of one specific route.
# The change might make the 'trips table becomes somewhat longer, but an analysis of trips len for opbranch=4 for each version shows that length of trips table was not significantly too different.

In [60]:
for version in VERSIONS:
    print(version, len(DFK[version, '4', 'trips']))
        

20220403_025040 190121
20220715_191941 81084
20230805_030129 54201
20231021_105623 97470
20231030_224547 97470
20231107_012112 106648
20231113_155410 82154
20231122_000348 83132
20231130_234004 80758
20231202_053358 105405
20231225_101846 195592
20240130_030040 137615
20240229_224711 124318
